# Run Conditional DCorr

Conditional DCorr is a version of conditional independence testing that has also shown properties of "uniform" consistency for hypothesis testing.

We want to compute results here to compare against CoMIGHT

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import math
from collections import defaultdict
from itertools import product

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from hyppo.conditional import ConditionalDcorr
from joblib import Parallel, delayed
from sklearn.model_selection import StratifiedShuffleSplit

from sktree.stats.utils import (
    METRIC_FUNCTIONS,
    POSITIVE_METRICS,
    POSTERIOR_FUNCTIONS,
    REGRESSOR_METRICS,
    _compute_null_distribution_coleman,
    _non_nan_samples,
)

seed = 12345
rng = np.random.default_rng(seed)

# Parallelizable function for any simulation

In [3]:
def _run_parallel_posterior_sim(
    idx,
    n_samples,
    n_features,
    class_probs,
    seed,
    n_features_2,
    test_size,
    max_fpr,
    sim_type,
):
    n_jobs = 1
    results = defaultdict(list)
    rng = np.random.default_rng(seed)
    n_features_ends = [100, None]

    if sim_type == "confounder":
        npy_data = np.load(
            f"/Users/adam2392/Desktop/cancer/confounder/confounder_{idx}.npz"
        )
    elif sim_type == "collider":
        npy_data = np.load(
            f"/Users/adam2392/Desktop/cancer/collider/collider_{idx}.npz"
        )
    elif sim_type == "mediator":
        npy_data = np.load(
            f"/Users/adam2392/Desktop/cancer/mediator/mediator_{idx}.npz"
        )
    elif sim_type == "direct-indirect":
        npy_data = np.load(
            f"/Users/adam2392/Desktop/cancer/direct-indirect/direct-indirect_{idx}.npz"
        )
    elif sim_type == "independent":
        npy_data = np.load(
            f"/Users/adam2392/Desktop/cancer/independent/independent_{idx}.npz"
        )

    X = npy_data["X"]
    y = npy_data["y"]
    # print(X.shape, y.shape)

    X = X[:, : 100 + n_features_2]
    if n_samples < X.shape[0]:
        cv = StratifiedShuffleSplit(n_splits=1, train_size=n_samples)
        for train_idx, _ in cv.split(X, y):
            continue
        X = X[train_idx, :]
        y = y[train_idx, ...].squeeze()
    assert len(X) == len(y)
    assert len(y) == n_samples
    n_features_ends[1] = X.shape[1]

    posteriors_dict = dict()

    # now compute the pvalue when shuffling X2
    covariate_index = np.arange(n_features_ends[0], n_features_ends[1])

    # estimate (conditional) mutual information using KSG
    Z = X[:, covariate_index]
    mask_array = np.ones(X.shape[1])
    mask_array[covariate_index] = 0
    mask_array = mask_array.astype(bool)
    X_minus_Z = X[:, mask_array]

    cdcorr = ConditionalDcorr(bandwidth="scott")

    print(Z, np.var(Z))
    # print(X_minus_Z, np.var(X_minus_Z))
    print(Z.shape, X_minus_Z.shape, y.shape)
    try:
        cdcorr_stat, cdcorr_pvalue = cdcorr.test(
            x=X_minus_Z.copy().astype(np.float64),
            y=y.copy().astype(np.float64),
            z=Z.copy().astype(np.float64),
        )
    except Exception as e:
        errmsg = f"{idx, n_samples, n_features, n_features_2, np.var(Z), X_minus_Z.shape, y.shape, Z.shape}"
        e.args += (errmsg,)
        raise (e)

    np.savez(
        f"./varying-samples/{sim_type}/conddcorr_{n_samples}_{n_features_2}_{idx}.npz",
        n_samples=n_samples,
        n_features_2=n_features_2,
        y_true=y,
        cdcorr_state=cdcorr_stat,
        cdcorr_pvalue=cdcorr_pvalue,
    )
    # results["cdcorr_pvalue_x2"].append(cdcorr_pvalue)
    # results["cdcorr_stat_x2"].append(cdcorr_stat)

    # results["mvrf_posteriors"].append(comight_posteriors_x2)
    # results["mvrf_null_posteriors"].append(comight_null_posteriors_x2)
    return results

# PValue Computations Varying Dimensionality of X2

In [4]:
# NOTE: increase this when running for full figure to 100
n_repeats = 100

In [5]:
# number of features in the first view
n_features = 10
noise_dims = 90

n_samples = 256
max_features = 0.3
n_jobs = -1
test_size = 0.2

max_fpr = 0.1

# number of features in the second view
pows = np.arange(2, 13, dtype=int)
n_features_2_list = [0] + [2**pow for pow in pows]

n_features_2_list = [2**pow for pow in pows]
print(n_features_2_list)

class_probs = [0.5, 0.5]

[4, 8, 16, 32, 64, 128, 256, 512, 1024, 2048, 4096]


In [6]:
print(n_features_2_list)

[4, 8, 16, 32, 64, 128, 256, 512, 1024, 2048, 4096]


## Independent Results

In [7]:
_results_ind = Parallel(n_jobs=-1)(
    delayed(_run_parallel_posterior_sim)(
        idx_,
        n_samples,
        n_features,
        class_probs,
        seed,
        n_features_2_,
        test_size,
        max_fpr,
        "independent",
    )
    for (idx_, n_features_2_) in product(range(n_repeats), n_features_2_list)
)

# {idx, n_samples, n_features, n_features_2}

/Users/adam2392/Documents/hyppo/hyppo/conditional/cdcorr.py:217: RuntimeWarning: overflow encountered in power
  denom = np.power(2 * np.pi, d / 2) * np.power(bandwidth_.prod(), 0.5)
/Users/adam2392/Documents/hyppo/hyppo/conditional/cdcorr.py:217: RuntimeWarning: overflow encountered in power
  denom = np.power(2 * np.pi, d / 2) * np.power(bandwidth_.prod(), 0.5)
/Users/adam2392/Documents/hyppo/hyppo/conditional/cdcorr.py:217: RuntimeWarning: overflow encountered in power
  denom = np.power(2 * np.pi, d / 2) * np.power(bandwidth_.prod(), 0.5)
/Users/adam2392/Documents/hyppo/hyppo/conditional/cdcorr.py:217: RuntimeWarning: overflow encountered in power
  denom = np.power(2 * np.pi, d / 2) * np.power(bandwidth_.prod(), 0.5)
/Users/adam2392/Documents/hyppo/hyppo/conditional/cdcorr.py:217: RuntimeWarning: overflow encountered in power
  denom = np.power(2 * np.pi, d / 2) * np.power(bandwidth_.prod(), 0.5)
/Users/adam2392/Documents/hyppo/hyppo/conditional/cdcorr.py:217: RuntimeWarning: over

[[ 2.20954603  0.12278706 -1.64224312 ...  0.4963114  -1.05766931
   1.13484185]
 [-0.02270628 -0.22531537  0.32086887 ...  0.48166069  1.60464735
  -0.76164337]
 [-0.6041721  -0.9451815  -1.0658785  ... -0.41822845 -0.16688235
   0.88128755]
 ...
 [-0.27811215  1.42828143 -0.19839126 ...  0.1169345   1.09908561
   0.23387404]
 [ 0.28990372 -0.48935689  0.55366137 ... -1.89206814  1.54556472
  -0.23755623]
 [-0.12557662  0.52325242 -1.0306507  ... -0.1511866  -0.09191958
  -1.02774067]] 0.9911961061658242
(256, 128) (256, 100) (256,)
[[ 0.974303   -0.47718166  0.21436605 ...  0.6261333   2.13917784
  -0.08569217]
 [ 0.78196064 -0.04991388 -0.06910147 ... -0.99782345 -0.52851396
  -0.08443322]
 [-0.27085029  1.40577703  1.4399205  ...  0.17637968  2.77876569
   0.95798701]
 ...
 [ 0.63364257 -1.45473417  0.50738316 ... -0.21697808 -0.89674537
  -1.39385544]
 [ 0.15138162  0.83088596  1.70119977 ... -0.53403395  0.15789014
  -2.17717262]
 [ 0.81729085  1.85496692 -0.75116705 ...  0.58246

/Users/adam2392/Documents/hyppo/hyppo/conditional/cdcorr.py:217: RuntimeWarning: overflow encountered in power
  denom = np.power(2 * np.pi, d / 2) * np.power(bandwidth_.prod(), 0.5)


[[-0.52657324 -0.60340806  1.15873376 ... -1.23968685  0.03564921
  -0.0702338 ]
 [ 1.28978219  1.01647865 -0.49440615 ... -0.22367843  1.16498566
  -1.36084559]
 [ 0.40050159 -1.06783523 -0.69241061 ...  0.05619994 -1.22922165
   0.33236605]
 ...
 [-0.40570296 -1.40627019 -0.36260142 ... -1.70264126  0.68237504
   0.33724757]
 [-1.51455633  0.24754675  0.70503185 ... -0.75411635  0.22074858
   1.01038625]
 [ 1.30678646 -0.41658768 -1.38859262 ... -1.84110569  1.50375787
   1.08141084]] 0.9958593932309547
(256, 256) (256, 100) (256,)
[[ 0.47924264  0.82558499 -0.44337661 ...  1.92240857  0.61349553
   1.87373436]
 [-0.58713263  0.5688483   1.21519488 ...  0.20106203 -0.61591228
  -0.49989105]
 [-1.46017309 -1.2383824  -0.11400582 ...  1.37978389 -0.23280354
   0.18083541]
 ...
 [ 0.60781986  1.29105276 -1.53318952 ... -0.85261942  0.33231265
   0.18351325]
 [-0.84282511 -1.18383023  0.74461981 ... -0.05004345  1.04261211
   0.10552481]
 [ 0.64565908 -0.13972369 -0.6941621  ...  0.95947

/Users/adam2392/Documents/hyppo/hyppo/conditional/cdcorr.py:217: RuntimeWarning: overflow encountered in power
  denom = np.power(2 * np.pi, d / 2) * np.power(bandwidth_.prod(), 0.5)


...

/Users/adam2392/Documents/hyppo/hyppo/conditional/cdcorr.py:217: RuntimeWarning: overflow encountered in power
  denom = np.power(2 * np.pi, d / 2) * np.power(bandwidth_.prod(), 0.5)
/Users/adam2392/Documents/hyppo/hyppo/conditional/cdcorr.py:217: RuntimeWarning: overflow encountered in power
  denom = np.power(2 * np.pi, d / 2) * np.power(bandwidth_.prod(), 0.5)
/Users/adam2392/Documents/hyppo/hyppo/conditional/cdcorr.py:217: RuntimeWarning: overflow encountered in power
  denom = np.power(2 * np.pi, d / 2) * np.power(bandwidth_.prod(), 0.5)


[[ 0.26959054 -1.70250773  0.12055811 ... -0.02327351 -1.16925448
  -1.60010689]
 [-0.40985523 -1.98246413  0.09443079 ...  2.69179621  1.45927563
  -0.10542513]
 [ 0.02971244 -0.38693027 -0.31735919 ... -0.19828567  0.15667692
  -0.11136844]
 ...
 [-0.17709083  0.16903973 -0.46931153 ... -2.10690083 -0.06356941
  -1.26317904]
 [ 1.68875686  1.08338614 -0.50962533 ... -0.18309573 -0.249074
   0.24088051]
 [ 1.31544476 -1.20442785  0.60382206 ... -0.48264503  1.19656931
   0.30513807]] 1.0014721149712948
(256, 128) (256, 100) (256,)
[[-2.20253874 -0.56403776 -0.61446397 ... -0.2198107   1.58599883
  -0.72981591]
 [-0.27642956 -0.63284408  0.1872734  ... -1.6791488   0.36370086
   0.08760503]
 [-0.06443925 -0.64106567 -0.91582347 ... -0.81557311 -0.75702263
   0.55393525]
 ...
 [ 1.66599698 -0.14843858  0.04500515 ...  0.74217297 -0.05388704
   0.64784273]
 [ 1.34951425  0.02115402 -0.58377283 ...  0.80597328 -1.0812892
   1.29859541]
 [-0.20445753  0.16129563 -1.27557359 ...  0.17179883

/Users/adam2392/Documents/hyppo/hyppo/conditional/cdcorr.py:217: RuntimeWarning: overflow encountered in power
  denom = np.power(2 * np.pi, d / 2) * np.power(bandwidth_.prod(), 0.5)
/Users/adam2392/Documents/hyppo/hyppo/conditional/cdcorr.py:217: RuntimeWarning: overflow encountered in power
  denom = np.power(2 * np.pi, d / 2) * np.power(bandwidth_.prod(), 0.5)
/Users/adam2392/Documents/hyppo/hyppo/conditional/cdcorr.py:217: RuntimeWarning: overflow encountered in power
  denom = np.power(2 * np.pi, d / 2) * np.power(bandwidth_.prod(), 0.5)


[[ 0.55357081  0.67206806  0.00940884 ... -0.6970075  -0.7911303
   1.7320029 ]
 [-0.23900507  0.61443475  1.36538018 ...  0.86398684 -0.42356679
  -1.29194764]
 [ 1.10732644 -0.22330036 -0.43552134 ... -1.13053425 -0.83380431
   0.2342036 ]
 ...
 [ 0.88681979 -0.23030201  0.91933864 ...  0.02771491  0.18109428
  -0.43109663]
 [-0.93779707  1.08946084 -1.28453487 ...  1.51000194 -0.20732916
  -0.12235468]
 [-1.864201    0.75772345  0.22131766 ... -1.61846072 -0.20573482
   0.76419935]] 0.9929254294359116
(256, 256) (256, 100) (256,)
[[ 1.46179824 -0.13230842 -0.14582547 ... -0.35418538  1.43165889
   0.91802078]
 [-0.09902176  0.30089443  0.96625314 ...  0.44176402 -0.846547
  -1.74872377]
 [ 0.16891763  0.51909523 -1.83318035 ...  0.78714    -0.52402386
  -0.03543736]
 ...
 [ 0.83002812 -0.72635547  2.37799088 ... -0.32369495 -1.45105493
   2.53910682]
 [-0.88410912  0.77652301 -1.0948836  ...  1.04432278  0.36443406
  -2.18184288]
 [ 1.7721003   0.88438483  1.2739733  ... -0.29346127

/Users/adam2392/Documents/hyppo/hyppo/conditional/cdcorr.py:217: RuntimeWarning: overflow encountered in power
  denom = np.power(2 * np.pi, d / 2) * np.power(bandwidth_.prod(), 0.5)
/Users/adam2392/Documents/hyppo/hyppo/conditional/cdcorr.py:217: RuntimeWarning: overflow encountered in power
  denom = np.power(2 * np.pi, d / 2) * np.power(bandwidth_.prod(), 0.5)
/Users/adam2392/Documents/hyppo/hyppo/conditional/cdcorr.py:217: RuntimeWarning: overflow encountered in power
  denom = np.power(2 * np.pi, d / 2) * np.power(bandwidth_.prod(), 0.5)


## Confounder Results

In [9]:
print("done")

done


In [10]:
_results_confounder = Parallel(n_jobs=-1)(
    delayed(_run_parallel_posterior_sim)(
        idx_,
        n_samples,
        n_features,
        class_probs,
        seed,
        n_features_2_,
        test_size,
        max_fpr,
        "confounder",
    )
    for (idx_, n_features_2_) in product(range(n_repeats), n_features_2_list)
)
# {idx, n_samples, n_features, n_features_2}

[[ 0.69860219  0.37501992 -1.92950143 ... -0.905097    0.25471235
  -1.76312482]
 [-0.48317953  1.39613195  1.49716422 ... -1.7462176  -0.18314323
   1.095924  ]
 [ 0.84364769  1.32447714 -0.13134295 ... -0.6705304   0.3327977
   2.03239257]
 ...
 [-0.22812356  0.17156737 -1.25165195 ... -0.03267069 -0.02128367
   0.06373303]
 [ 0.52815833 -1.80215407  1.02348332 ... -0.94692411  0.01531964
   1.54460441]
 [-0.59239972  1.40809558 -0.72562595 ... -1.00971275 -0.62037055
   0.31137712]] 0.9688006458993984
(256, 64) (256, 100) (256,)
[[-1.4822823  -0.27201538 -0.22809511 ...  0.4197406  -0.68726661
  -1.35602443]
 [ 0.57614976 -0.41344456 -1.44199079 ...  0.20370536  0.7848533
   1.98807818]
 [-0.32234524 -0.76291492  0.5066942  ... -0.30179647  0.63739647
  -0.91757485]
 ...
 [ 0.64993265  0.66619659 -0.77186554 ...  0.22657242 -1.86320856
  -0.82094898]
 [ 0.18254134 -0.91232347  1.02365771 ... -1.73276923 -0.24501561
  -0.3433405 ]
 [-0.17387559 -1.88769903 -0.24186695 ... -0.0133312 

/Users/adam2392/Documents/hyppo/hyppo/conditional/cdcorr.py:217: RuntimeWarning: overflow encountered in power
  denom = np.power(2 * np.pi, d / 2) * np.power(bandwidth_.prod(), 0.5)
/Users/adam2392/Documents/hyppo/hyppo/conditional/cdcorr.py:217: RuntimeWarning: overflow encountered in power
  denom = np.power(2 * np.pi, d / 2) * np.power(bandwidth_.prod(), 0.5)
/Users/adam2392/Documents/hyppo/hyppo/conditional/cdcorr.py:217: RuntimeWarning: overflow encountered in power
  denom = np.power(2 * np.pi, d / 2) * np.power(bandwidth_.prod(), 0.5)
/Users/adam2392/Documents/hyppo/hyppo/conditional/cdcorr.py:217: RuntimeWarning: overflow encountered in power
  denom = np.power(2 * np.pi, d / 2) * np.power(bandwidth_.prod(), 0.5)
/Users/adam2392/Documents/hyppo/hyppo/conditional/cdcorr.py:217: RuntimeWarning: overflow encountered in power
  denom = np.power(2 * np.pi, d / 2) * np.power(bandwidth_.prod(), 0.5)
/Users/adam2392/Documents/hyppo/hyppo/conditional/cdcorr.py:217: RuntimeWarning: over

[[ 1.24288881  2.14500768 -0.25738752 ... -0.67193456  2.46090409
   0.5341129 ]
 [ 0.55392777  0.00801619 -0.38727234 ...  0.87621257 -1.10143492
  -0.93630161]
 [ 0.09359257 -1.8026496  -0.43908896 ... -1.08857626 -1.20854789
  -1.45292111]
 ...
 [-2.99554475 -0.45830399  0.34012481 ...  1.10467049  1.13760105
  -0.85430107]
 [ 0.40200619 -1.97344679  1.24503375 ... -0.66211788 -1.45304489
  -0.07268838]
 [-0.24427351  0.0565823  -0.39322393 ... -0.66206438 -0.22814404
   1.4708737 ]] 1.0014850631696746
(256, 128) (256, 100) (256,)
[[ 0.34536926 -0.7260329  -1.66904171 ...  0.86770721  0.85074271
   1.62288334]
 [ 0.24879151 -0.84362303 -0.91340673 ...  0.05098101  1.64894311
   0.68730976]
 [-0.58937419  1.60375299  1.7387922  ...  0.64722149  1.18573077
  -2.60252626]
 ...
 [ 1.8323723   1.05669668 -2.2120196  ...  0.40395701 -1.51259429
   0.38523577]
 [ 0.35948763  0.43619202  0.71778975 ... -0.7274238  -0.64555638
   0.01008674]
 [-0.52756424 -1.32409429  0.4783707  ... -0.84568

/Users/adam2392/Documents/hyppo/hyppo/conditional/cdcorr.py:217: RuntimeWarning: overflow encountered in power
  denom = np.power(2 * np.pi, d / 2) * np.power(bandwidth_.prod(), 0.5)
/Users/adam2392/Documents/hyppo/hyppo/conditional/cdcorr.py:217: RuntimeWarning: overflow encountered in power
  denom = np.power(2 * np.pi, d / 2) * np.power(bandwidth_.prod(), 0.5)
/Users/adam2392/Documents/hyppo/hyppo/conditional/cdcorr.py:217: RuntimeWarning: overflow encountered in power
  denom = np.power(2 * np.pi, d / 2) * np.power(bandwidth_.prod(), 0.5)
/Users/adam2392/Documents/hyppo/hyppo/conditional/cdcorr.py:217: RuntimeWarning: overflow encountered in power
  denom = np.power(2 * np.pi, d / 2) * np.power(bandwidth_.prod(), 0.5)


[[ 0.30007916 -0.47797042 -1.06110286 ... -0.14416984 -1.11997555
   0.01768794]
 [ 0.51499462  0.84066717 -0.43561975 ... -1.65810715 -1.0409949
   0.23724737]
 [ 0.09480095 -2.02538072 -0.74197671 ... -0.6001155   0.29422337
   1.43901062]
 ...
 [-0.42279735 -0.44827256  1.1519032  ...  0.38567797 -0.16968865
  -0.78354119]
 [-0.3932423   0.79171432 -0.81357235 ...  1.15651946  1.69260817
  -0.22318245]
 [ 0.01007324  1.32215141 -0.07058638 ... -0.11459466 -0.64151624
   1.64314177]] 0.9990048235121081
(256, 2048) (256, 100) (256,)
[[ 0.80497292  0.65354865 -0.70234682 ... -1.14551644  0.51532891
   1.34345974]
 [ 0.0659526   0.24798963 -1.17393409 ...  0.30940347 -0.71315227
   0.04958806]
 [ 0.77446647  0.26092312  1.09605871 ...  0.33929347 -0.56112621
  -0.08375187]
 ...
 [-1.17810996  0.61422967 -0.43464061 ... -1.1297454  -0.08006143
  -2.11135906]
 [ 0.26395729  0.11404312  0.91923839 ...  0.09129608  1.03413488
   0.77289437]
 [ 1.67657006 -1.1311996  -0.40744521 ... -0.62376

...

[[ 4.09658216e+00  1.97665422e+00 -1.89203775e+00 ...  2.07086987e+00
   1.84560040e-02  1.22426693e+00]
 [ 1.44969720e+00  2.77627757e+00  5.59820828e-02 ... -4.09554317e-01
  -8.14204585e-01 -2.81336224e-01]
 [-1.79040121e+00 -4.18858794e+00 -3.46247809e+00 ...  7.36739945e-01
   1.03408223e+00  2.52853004e-01]
 ...
 [ 2.35649463e+00  1.14769358e+01 -1.95030369e+00 ...  1.12071823e+00
   1.02428323e+00 -3.81470583e-02]
 [ 5.77384077e+00  6.89306581e+00  2.10015315e+01 ...  2.75172557e+00
   2.76478304e+00  7.57807661e-01]
 [ 5.83866224e+00  2.36319188e+00  2.42907388e+00 ...  6.23062455e-01
   6.35680648e-01 -7.16377800e-01]] 32489.82434910617
(256, 256) (256, 100) (256,)
[[ 2.28879975e+00  2.08607317e+00  1.53624784e+01 ... -2.89045149e+00
  -3.37536226e-01  5.57971849e-01]
 [-1.31463806e+00  1.79642927e+00  1.22182668e+01 ... -3.20157501e-02
   1.31572030e+00  2.47396355e+00]
 [ 5.88021973e+00  2.22479192e-01  1.20088117e+00 ...  8.05056435e-01
   3.01211835e+00 -1.67878705e+00]
 .

/Users/adam2392/Documents/hyppo/hyppo/conditional/cdcorr.py:217: RuntimeWarning: overflow encountered in power
  denom = np.power(2 * np.pi, d / 2) * np.power(bandwidth_.prod(), 0.5)
/Users/adam2392/Documents/hyppo/hyppo/conditional/cdcorr.py:217: RuntimeWarning: overflow encountered in power
  denom = np.power(2 * np.pi, d / 2) * np.power(bandwidth_.prod(), 0.5)
/Users/adam2392/Documents/hyppo/hyppo/conditional/cdcorr.py:217: RuntimeWarning: overflow encountered in power
  denom = np.power(2 * np.pi, d / 2) * np.power(bandwidth_.prod(), 0.5)
/Users/adam2392/Documents/hyppo/hyppo/conditional/cdcorr.py:217: RuntimeWarning: overflow encountered in power
  denom = np.power(2 * np.pi, d / 2) * np.power(bandwidth_.prod(), 0.5)
/Users/adam2392/Documents/hyppo/hyppo/conditional/cdcorr.py:217: RuntimeWarning: overflow encountered in power
  denom = np.power(2 * np.pi, d / 2) * np.power(bandwidth_.prod(), 0.5)
/Users/adam2392/Documents/hyppo/hyppo/conditional/cdcorr.py:217: RuntimeWarning: over

[[ 2.57350422  2.05502221 -2.64425397 ...  0.55023187 -0.32916926
   2.10228439]
 [19.75308172 13.55959101  2.5953823  ...  0.2082325   1.16196352
   1.60736469]
 [ 0.14883658  1.58542642 -0.03053576 ...  1.39483345 -0.98346761
   0.1539194 ]
 ...
 [ 1.55963853 -1.22376299 -1.65276581 ...  0.55546191 -0.23632313
   0.24065185]
 [-0.10151901  0.31093006  1.54943578 ... -1.58056168 -0.44585024
  -0.28332815]
 [ 2.46641299  0.2243282   1.36941829 ...  0.28241915  0.41786479
  -0.43984693]] 13.422970025565496
(256, 256) (256, 100) (256,)
[[13.49599847  0.05832109  1.43810677 ...  1.81570996 -2.27528409
   1.83237833]
 [13.79467694  2.68124973  2.0040745  ... -1.42712618  0.13469053
  -1.08397271]
 [ 3.44892348 -0.82320511  4.0590927  ...  0.12231475  0.35949197
   0.45211147]
 ...
 [ 1.67571918  2.76052305  2.33521542 ... -1.00200829  1.17928359
  -0.60026387]
 [ 1.22870438 -0.9984692   2.99747423 ... -2.25784101 -0.69198212
  -0.69038547]
 [ 3.54159832 -1.37887435 23.3407398  ...  0.60988

/Users/adam2392/Documents/hyppo/hyppo/conditional/cdcorr.py:217: RuntimeWarning: overflow encountered in power
  denom = np.power(2 * np.pi, d / 2) * np.power(bandwidth_.prod(), 0.5)


KeyboardInterrupt: 

## Collider Results

In [ ]:
_results_collider = Parallel(n_jobs=-1)(
    delayed(_run_parallel_posterior_sim)(
        idx_,
        n_samples,
        n_features,
        class_probs,
        seed,
        n_features_2_,
        test_size,
        max_fpr,
        "collider",
    )
    for (idx_, n_features_2_) in product(range(n_repeats), n_features_2_list)
)

## Direct-Indirect Results

In [ ]:
print(n_features_2_list)

In [ ]:
_results_directindirect = Parallel(n_jobs=-1)(
    delayed(_run_parallel_posterior_sim)(
        idx_,
        n_samples,
        n_features,
        class_probs,
        seed,
        n_features_2_,
        test_size,
        max_fpr,
        "direct-indirect",
    )
    for (idx_, n_features_2_) in product(range(n_repeats), n_features_2_list)
)